In [ ]:
from sklearn.preprocessing import StandardScaler
from visualization_fct import *

import itertools
from bokeh.io import output_notebook
output_notebook()
from bokeh.plotting import show  # output_file, save

from pomegranate import HiddenMarkovModel
from pomegranate import MultivariateGaussianDistribution
from pomegranate import State, Distribution

import matplotlib.pyplot as plt  # , mpld3

data = pd.read_csv("../asm_data/asm_data_for_ml.txt", sep='\t')
del data['MJD']
del data['error']
del data['errorA']
del data['errorB']
del data['errorC']
data['rateCA'] = data.rateC / data.rateA
data_thr = mask(data, 'orbit')  # rm too large values except for 'orbit'


np.random.seed(1)

X = np.c_[data_thr.orbit, data_thr.rate, data_thr.rateA, data_thr.rateB,
          data_thr.rateC, data_thr.rateCA]

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=3)
y = km.fit_predict(X)

In [ ]:
X_1 = X[y==0]
X_2 = X[y==1]
X_3 = X[y==2]
a = MultivariateGaussianDistribution.from_samples(X_1)
b = MultivariateGaussianDistribution.from_samples(X_2)
c = MultivariateGaussianDistribution.from_samples(X_3)
s1 = State(a, name="M1")
s2 = State(b, name="M2")
s3 = State(c, name="M3")

hmm = HiddenMarkovModel()
hmm.add_states(s1, s2, s3)
hmm.add_transition(hmm.start, s1, 0.4)
hmm.add_transition(hmm.start, s3, 0.3)
hmm.add_transition(hmm.start, s2, 0.3)

hmm.add_transition(s1, s1, 0.9)
hmm.add_transition(s1, s2, 0.1)

hmm.add_transition(s2, s1, 0.05)
hmm.add_transition(s2, s3, 0.05)
hmm.add_transition(s2, s2, 0.9)

hmm.add_transition(s3, s3, 0.9)
hmm.add_transition(s3, s2, 0.1)
hmm.bake()
hmm.fit(X)  # , weights=w) hmm does not support weights in pomegranate
preds = hmm.predict(X)
probs = hmm.predict_proba(X)

data_thr['preds'] = pd.Series(preds).astype("category")

color_key = ["red", "blue", "yellow", "grey", "black", "purple", "pink",
             "brown", "green", "orange"]  # Spectral9
color_key = color_key[:len(set(preds))+2]

covs = np.array([np.array(hmm.states[m].distribution.parameters[1])
                 for m in range(3)])
means = np.array([np.array(hmm.states[m].distribution.parameters[0])
                  for m in range(3)])

# transform cov for non-standardizeed data:
covs = np.array([np.dot(np.diag(np.sqrt(scaler.var_)),
                        np.dot(covs[j], np.diag(np.sqrt(scaler.var_))))
                 for j in range(covs.shape[0])])
means = np.array([scaler.inverse_transform(means[j].reshape(1, -1)).T
                  for j in range(means.shape[0])])

In [ ]:
# single plot rateCA vs rate with predicted classes and ellipses:
x = 5
y = 1
covs_xy = [covs[j][[x, y]][:, [x, y]] for j in range(len(covs))]
means_xy = [means[j][[x, y]] for j in range(len(covs))]

    # # # #uncommment if log
    # for j in range(len(covs)):
    #     covs_xy[j][0] = np.exp(covs_xy[j][0]) - 1
    #     means_xy[j][0] = np.exp(means_xy[j][0]) - 1

single_plot = bokeh_datashader_plot(data_thr, covs=covs_xy, means=means_xy,
                                    x_name='rateCA',
                                    y_name='rate',
                                    plot_width=900, plot_height=300,
                                    pixel_width=3000, pixel_height=1000,
                                    spread=True, color_key=color_key)
show(single_plot)

In [ ]:
T = np.exp(hmm.dense_transition_matrix())[:3, :3]
%matplotlib notebook
plt.figure()
plt.imshow(T, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()
for i, j in itertools.product(range(T.shape[0]), range(T.shape[1])):
        plt.text(j, i, int(T[i, j]*100),
                 horizontalalignment="center",
                 color="white" if T[i, j] > 0.5 else "black")
plt.title('log_likelihood:%0.3f' % hmm.log_probability(X))
#["red", "blue", "yellow", "grey", "black", "purple", "pink", "brown", "green"]

In [ ]:
import matplotlib.animation as animation

fig, ax = plt.subplots()

XX = np.c_[data_thr.rateCA, data_thr.rate]
XX = XX[np.array(data_thr.rateCA < 5) * np.array(data_thr.rate < 80)]

#line, = ax.plot(En[0, :], X_flux[0, :])
line, = ax.plot(XX[:, 0], XX[:, 1], '.')

for n_comp in range(len(covs_xy)):
    cov = covs_xy[n_comp]
    mean = means_xy[n_comp]
    v, w = np.linalg.eigh(cov)
    e0 = w[0] / np.linalg.norm(w[0])
    e1 = w[1] / np.linalg.norm(w[1])
    t = np.linspace(0, 2 * np.pi, 10000)
    # 4.605 corresponds to 90% quantile:
    a = (mean[0]
         + np.sqrt(4.605 * v[0]) * np.cos(t) * e0[0]
         + np.sqrt(4.605 * v[1]) * np.sin(t) * e1[0])
    b = (mean[1]
         + np.sqrt(4.605 * v[0]) * np.cos(t) * e0[1]
         + np.sqrt(4.605 * v[1]) * np.sin(t) * e1[1])

    ax.plot(a, b, color=color_key[n_comp])


def animate(i):
    line.set_xdata(XX[10*i:10*(i+1), 0])  # update the data
    line.set_ydata(XX[10*i:10*(i+1), 1])   # update the data
    line.set_color(color_key[preds[10*i]])
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(1000, 9300), #init_func=init,
                              interval=1, blit=True)
#plt.show()
